In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.cm import rainbow
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pickle
import shap
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, roc_curve, auc, precision_recall_curve, 
                             confusion_matrix, precision_score, recall_score, f1_score, 
                             roc_auc_score, classification_report)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# 加载数据
dataset = pd.read_csv('C:/Users/Administrator/OneDrive/help/评分模型/dataset/dataset.CSV', encoding="ANSI")

# 显示数据基本信息
print("数据集基本信息：")
print(dataset.info())

print("\n缺失值统计：")
print(dataset.isnull().sum())

# 选择需要填充的数值列
numeric_columns = dataset.select_dtypes(include=[np.number]).columns
print(numeric_columns)

# 加载数据
dataset = pd.read_csv('C:/Users/Administrator/OneDrive/help/评分模型/dataset/dataset.CSV', encoding="ANSI")

# 显示数据基本信息
print("数据集基本信息：")
print(dataset.info())

print("\n缺失值统计：")
print(dataset.isnull().sum())

# 选择需要填充的数值列
numeric_columns = dataset.select_dtypes(include=[np.number]).columns
print(numeric_columns)

# 使用IterativeImputer填充
it_imputer = IterativeImputer(estimator=RandomForestRegressor(), random_state=0)
dataset_imputed_it = dataset.copy()
dataset_imputed_it[numeric_columns] = it_imputer.fit_transform(dataset[numeric_columns])

print("\n使用IterativeImputer填充后的数据集：")
print(dataset_imputed_it.info())
print(dataset_imputed_it.isnull().sum())

# 使用KNNImputer填充
knn_imputer = KNNImputer(n_neighbors=5)
dataset_imputed_knn = dataset.copy()
dataset_imputed_knn[numeric_columns] = knn_imputer.fit_transform(dataset[numeric_columns])

print("\n使用KNNImputer填充后的数据集：")
print(dataset_imputed_knn.info())
print(dataset_imputed_knn.isnull().sum())

# 保存填充后的数据集
dataset_imputed_it.to_csv('dataset_imputed_iterative.csv', index=False, encoding='ANSI')
dataset_imputed_knn.to_csv('dataset_imputed_knn.csv', index=False, encoding='ANSI')

print("\n填充后的数据集已保存为 'dataset_imputed_iterative.csv' 和 'dataset_imputed_knn.csv'")

In [ ]:
dataset = pd.read_csv('C:/Users/Administrator/OneDrive/help/评分模型/dataset_imputed_iterative.CSV',encoding = "ANSI")

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset["DGF"].describe()
dataset["DGF"].unique()
print(dataset.corr()["DGF"].abs().sort_values(ascending=False))

In [ ]:
# Cell 2: 加载和预处理数据
# Load dataset
df = dataset  # Make sure you have a DataFrame named 'dataset'

# Separate features and target variable
X = df.drop('DGF', axis=1)
y = df['DGF']

print(f"Positive class percentage: {y.mean()*100:.2f}%")

# Apply RandomUnderSampler to the entire dataset
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

print(f"Positive class percentage after resampling: {y_resampled.mean()*100:.2f}%")

# Standardizing the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)

# 这个cell加载数据集，分离特征和目标变量，应用随机欠采样来平衡类别，
# 并对特征进行标准化。它还打印了正类的百分比，以显示重采样前后的类别分布。

In [ ]:
# Cell 3: 定义优化函数
def optimize_logistic_regression(X, y):
    param_space = {
        'C': Real(1e-6, 1e+6, prior='log-uniform'),
        'penalty': Categorical(['l1', 'l2'])
    }
    
    optimizer = BayesSearchCV(
        LogisticRegression(random_state=42, solver='liblinear'),
        param_space,
        n_iter=20,
        cv=4,
        n_jobs=-1,
        random_state=42,
        scoring='roc_auc'
    )
    
    optimizer.fit(X, y)
    return optimizer.best_estimator_

def optimize_random_forest(X, y):
    param_space = {
        'n_estimators': Integer(10, 200),
        'max_depth': Integer(2, 20),
        'min_samples_split': Integer(2, 10),
        'min_samples_leaf': Integer(1, 10)
    }
    
    optimizer = BayesSearchCV(
        RandomForestClassifier(random_state=42),
        param_space,
        n_iter=20,
        cv=4,
        n_jobs=-1,
        random_state=42,
        scoring='roc_auc'
    )
    
    optimizer.fit(X, y)
    return optimizer.best_estimator_

# 这个cell定义了两个优化函数，分别用于逻辑回归和随机森林模型。
# 这些函数使用贝叶斯搜索来优化模型的超参数。

In [ ]:
# Cell 4: 初始化模型
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'GaussianNB': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(random_state=42, probability=True),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Neural Network': MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1),
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(random_state=42)
}

# 这个cell初始化了所有要测试的模型，并将它们存储在一个字典中。

In [ ]:
# Cell 5: 定义评估指标计算函数
def calculate_metrics(y_true, y_pred, y_pred_proba):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    precision_curve, recall_curve, _ = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = auc(recall_curve, precision_curve)
    return accuracy, precision, recall, f1, specificity, roc_auc, pr_auc, fpr, tpr, precision_curve, recall_curve

# 这个cell定义了一个函数，用于计算各种评估指标，包括准确率、精确率、召回率、F1分数、
# 特异性、ROC AUC和PR AUC等。

In [ ]:
# Cell 6: 训练模型
# 假设dataset已经加载,并且'DGF'是目标变量
X = dataset.drop('DGF', axis=1)
y = dataset['DGF']

# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 初始化模型
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'GaussianNB': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(random_state=42, probability=True),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Neural Network': MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1),
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(random_state=42)
}

# 70%训练, 30%测试
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 创建一个图表来存储所有ROC曲线
plt.figure(figsize=(12, 8))

# 为每个模型计算并绘制ROC曲线
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 计算准确率
    accuracy = accuracy_score(y_test, y_pred)
    
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')
    
    print(f"{name} - 70-30 split ROC-AUC: {roc_auc:.4f}, Accuracy: {accuracy:.4f}")

plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.savefig(f"AUC.pdf", format="pdf", bbox_inches='tight')
plt.show()


In [ ]:
# Cell 7: cross-validation
# 10倍交叉验证
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 存储结果的字典
results = {
    'Accuracy': [],
    'ROC-AUC': []
}

# 为每个模型进行交叉验证
for name, model in models.items():
    accuracies = cross_val_score(model, X_scaled, y, cv=cv, scoring='accuracy')
    roc_aucs = cross_val_score(model, X_scaled, y, cv=cv, scoring='roc_auc')
    
    mean_accuracy = np.mean(accuracies)
    mean_roc_auc = np.mean(roc_aucs)
    
    results['Accuracy'].append(mean_accuracy)
    results['ROC-AUC'].append(mean_roc_auc)
    
    print(f"{name}:")
    print(f"  10-fold CV Mean Accuracy: {mean_accuracy:.4f}")
    print(f"  10-fold CV Mean ROC-AUC: {mean_roc_auc:.4f}")

# 绘制折线图
plt.figure(figsize=(12, 6))

# 准确率折线图
plt.subplot(1, 2, 1)
plt.plot(list(models.keys()), results['Accuracy'], marker='o')
plt.title('Mean Accuracy for Different Models')
plt.xlabel('Models')
plt.ylabel('Mean Accuracy')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1)
plt.grid(True)

# ROC-AUC折线图
plt.subplot(1, 2, 2)
plt.plot(list(models.keys()), results['ROC-AUC'], marker='o')
plt.title('Mean ROC-AUC for Different Models')
plt.xlabel('Models')
plt.ylabel('Mean ROC-AUC')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1)
plt.grid(True)

plt.tight_layout()
plt.savefig(f"10-fold cross validation.pdf", format="pdf", bbox_inches='tight')
plt.show()

In [ ]:
# Cell 8: 保存训练好的模型
def ensemble_predict(models, X):
    predictions = []
    for model in models.values():
        if hasattr(model, 'predict_proba'):
            pred = model.predict_proba(X)[:, 1]
        else:
            pred = model.predict(X)
        predictions.append(pred)
    return np.mean(predictions, axis=0)

# Cell 10: 保存集成模型

import pickle
import os

# 确保存在保存模型的目录
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

# 保存各个模型
for model_name, model in models.items():
    filename = f'saved_models/{model_name.replace(" ", "_").lower()}.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    print(f'{model_name} 已保存为 {filename}')

# 保存集成函数和模型字典
ensemble_package = {
    'ensemble_function': ensemble_predict,
    'models': models
}

with open('saved_models/ensemble_model.pkl', 'wb') as file:
    pickle.dump(ensemble_package, file)

print("集成模型已保存为 saved_models/ensemble_model.pkl")

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, confusion_matrix
# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 初始化模型
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'GaussianNB': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(random_state=42, probability=True),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Neural Network': MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, max_iter=1000),
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(random_state=42, verbose=-1)  # 设置verbose=-1来抑制警告
}

def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'specificity': make_scorer(specificity_score)
}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results = {}

for name, model in models.items():
    cv_results = cross_validate(model, X_scaled, y, cv=cv, scoring=scoring)
    
    results[name] = {
        'Accuracy': np.mean(cv_results['test_accuracy']),
        'ROC AUC Mean': np.mean(cv_results['test_roc_auc']),
        'ROC AUC Std': np.std(cv_results['test_roc_auc']),
        'Precision': np.mean(cv_results['test_precision']),
        'Recall': np.mean(cv_results['test_recall']),
        'F1 Score': np.mean(cv_results['test_f1']),
        'Specificity': np.mean(cv_results['test_specificity'])
    }

df_results = pd.DataFrame.from_dict(results, orient='index')
df_results = df_results[['ROC AUC Mean', 'ROC AUC Std', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Specificity']]

print("Model Performance Metrics (10-Fold Cross-Validation):")
print(df_results.to_string(float_format=lambda x: f"{x:.4f}"))

In [ ]:
# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 初始化三个模型
models = {
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(random_state=42, verbose=-1)
}

def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'specificity': make_scorer(specificity_score)
}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results = {}

for name, model in models.items():
    cv_results = cross_validate(model, X_scaled, y, cv=cv, scoring=scoring)
    
    results[name] = {
        'Accuracy': np.mean(cv_results['test_accuracy']),
        'ROC AUC Mean': np.mean(cv_results['test_roc_auc']),
        'ROC AUC Std': np.std(cv_results['test_roc_auc']),
        'Precision': np.mean(cv_results['test_precision']),
        'Recall': np.mean(cv_results['test_recall']),
        'F1 Score': np.mean(cv_results['test_f1']),
        'Specificity': np.mean(cv_results['test_specificity'])
    }

df_cv_results = pd.DataFrame.from_dict(results, orient='index')
df_cv_results = df_cv_results[['ROC AUC Mean', 'ROC AUC Std', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Specificity']]

print("模型性能指标（10倍交叉验证）:")
print(df_cv_results.to_string(float_format=lambda x: f"{x:.4f}"))

# 加载外部测试数据（单个样本）
test_df = pd.read_csv('C:/Users/Administrator/OneDrive/help/评分模型/dataset/test.CSV', encoding="ANSI")

# 分离特征和标准结果（如果有）
if test_df.shape[1] > len(X.columns):
    y_true = test_df.iloc[0, 0]  # 假设第一列是DGF的标准结果
    test_X = test_df.iloc[0, 1:]  # 其余列是特征
else:
    y_true = None
    test_X = test_df.iloc[0, :]  # 所有列都是特征

# 确保测试数据集的特征与训练数据集相同
test_X = test_X[X.columns]

# 标准化测试数据
test_X_scaled = scaler.transform(test_X.values.reshape(1, -1))

# 创建一个字典来存储测试集结果
test_results = {}

# 在外部测试集上评估模型
for name, model in models.items():
    print(f"\n{name} 对单个样本的预测结果:")
    
    # 使用全部训练数据重新训练模型
    model.fit(X_scaled, y)
    y_pred = model.predict(test_X_scaled)
    y_pred_proba = model.predict_proba(test_X_scaled)[0, 1]

    print(f"预测结果: {'正' if y_pred[0] == 1 else '负'}")
    print(f"预测概率 (正类别): {y_pred_proba:.4f}")

    if y_true is not None:
        print(f"实际结果: {'正' if y_true == 1 else '负'}")

    test_results[name] = {
        'Predicted': '正' if y_pred[0] == 1 else '负',
        'Probability': y_pred_proba
    }

# 创建测试集结果DataFrame
df_test_results = pd.DataFrame.from_dict(test_results, orient='index')

print("\n模型预测结果汇总:")
print(df_test_results.to_string(float_format=lambda x: f"{x:.4f}"))

In [ ]:
# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 初始化三个模型
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(random_state=42, verbose=-1)
}

def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'specificity': make_scorer(specificity_score)
}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results = {}

for name, model in models.items():
    cv_results = cross_validate(model, X_scaled, y, cv=cv, scoring=scoring)
    
    results[name] = {
        'Accuracy': np.mean(cv_results['test_accuracy']),
        'ROC AUC Mean': np.mean(cv_results['test_roc_auc']),
        'ROC AUC Std': np.std(cv_results['test_roc_auc']),
        'Precision': np.mean(cv_results['test_precision']),
        'Recall': np.mean(cv_results['test_recall']),
        'F1 Score': np.mean(cv_results['test_f1']),
        'Specificity': np.mean(cv_results['test_specificity'])
    }

df_cv_results = pd.DataFrame.from_dict(results, orient='index')
df_cv_results = df_cv_results[['ROC AUC Mean', 'ROC AUC Std', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Specificity']]

print("模型性能指标（10倍交叉验证）:")
print(df_cv_results.to_string(float_format=lambda x: f"{x:.4f}"))

# 加载外部测试数据
test_df = pd.read_csv('C:/Users/Administrator/OneDrive/help/评分模型/dataset/test2.CSV', encoding="ANSI")

# 分离特征和标准结果
y_true = test_df.iloc[:, 0]  # 假设第一列是DGF的标准结果
test_X = test_df.iloc[:, 1:]  # 其余列是特征

# 确保测试数据集的特征与训练数据集相同
test_X = test_X[X.columns]

# 标准化测试数据
test_X_scaled = scaler.transform(test_X)

# 创建一个字典来存储测试集结果
test_results = {}

# 创建一个图来绘制所有ROC曲线
plt.figure(figsize=(10, 8))

# 在外部测试集上评估模型
for name, model in models.items():
    print(f"\n{name} 在外部测试集上的表现:")
    
    # 使用全部训练数据重新训练模型
    model.fit(X, y)  # 这些模型不需要标准化的数据
    y_pred = model.predict(test_X)
    y_pred_proba = model.predict_proba(test_X)[:, 1]

    accuracy = accuracy_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    
    test_results[name] = {
        'Accuracy': accuracy,
        'ROC AUC': roc_auc,
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred),
        'Specificity': specificity_score(y_true, y_pred)
    }

    print(f"Accuracy: {accuracy:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred))

    # 绘制ROC曲线
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')

# 完成ROC曲线图
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - All Models')
plt.legend(loc="lower right")
plt.savefig(f"ROC-test.pdf", format="pdf", bbox_inches='tight')
plt.show()

# 创建测试集结果DataFrame
df_test_results = pd.DataFrame.from_dict(test_results, orient='index')
df_test_results = df_test_results[['ROC AUC', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Specificity']]

print("\n模型性能指标（测试集）:")
print(df_test_results.to_string(float_format=lambda x: f"{x:.4f}"))

In [ ]:
import shap
import matplotlib.pyplot as plt

# 确保 X 和 models 字典已经定义

shap_values_dict = {}

# 对XGBoost和LightGBM计算SHAP值
for model_name in ['XGBoost', 'LightGBM']:
    model = models[model_name]
    
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    
    # 对于XGBoost和LightGBM，shap_values通常已经是我们需要的形式
    # 但是为了保险起见，我们还是检查一下
    if isinstance(shap_values, list):
        shap_values = shap_values[1]  # 对于二分类问题，我们通常关注正类
    
    shap_values_dict[model_name] = shap_values

# 绘制详细的SHAP值图
for model_name, shap_values in shap_values_dict.items():
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X, show=False)
    plt.title(f'{model_name} SHAP Summary Plot')
    plt.tight_layout()
    plt.savefig(f"{model_name}_shap_summary.pdf", format="pdf", bbox_inches='tight')
    plt.show()

# 计算和打印平均绝对SHAP值
for model_name, shap_values in shap_values_dict.items():
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    feature_importance = pd.DataFrame(list(zip(X.columns, mean_abs_shap)), columns=['feature', 'importance'])
    feature_importance = feature_importance.sort_values('importance', ascending=False)
    
    print(f"\n{model_name} 特征重要性 (基于平均绝对SHAP值):")
    print(feature_importance.to_string(index=False))


